In [1]:
%pip install lifelines

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4119 sha256=4fe5eacb311f62d60a7eb70ecc63704a36acd802c38d9350289f06609c24b442
  Stored in directory: /Users/gymmnotjim/Library/Caches/pip/wheels/7e/16/46/9477f188924292d3bf1fb8fb42844201591abfc19b7ba6d868
Successfully built autograd-gamma
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [lifelines]/7 [lifelines]meta]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Example Usage
import pandas as pd
from lifelines.datasets import load_rossi
from lifelines import CoxPHFitter

# Load the sample dataset
df = load_rossi()
print(df.head())

# Create and fit the model
cph = CoxPHFitter()
cph.fit(df, duration_col='week', event_col='arrest')

# Print the summary of the model
cph.print_summary()

   week  arrest  fin  age  race  wexp  mar  paro  prio
0    20       1    0   27     1     0    0     1     3
1    17       1    0   18     1     0    0     1     8
2    25       1    0   19     0     1    0     1    13
3    52       0    1   23     1     1    1     1     1
4    52       0    0   19     0     1    0     1     3


<lifelines.CoxPHFitter: fitted with 432 total observations, 318 right-censored observations>
             duration col = 'week'
                event col = 'arrest'
      baseline estimation = breslow
   number of observations = 432
number of events observed = 114
   partial log-likelihood = -658.75
         time fit was run = 2025-10-04 14:45:06 UTC

---
           coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                  
fin       -0.38      0.68      0.19           -0.75           -0.00                0.47                1.00
age       -0.06      0.94      0.02           -0.10           -0.01                0.90                0.99
race       0.31      1.37      0.31           -0.29            0.92                0.75                2.50
wexp      -0.15      0.86      0.21           -0.57            0.27                0.57                1.30
mar       -0.43      0.65      0.38           -1.18            0.31                0.31                1.37
paro      -0.08      0.92      0.20           -0.47            0.30                0.63                1.35
prio       0.09      1.10      0.03            0.04            0.15                1.04                1.16

           cmp to     z      p  -log2(p)
covariate                               
fin          0.00 -1.98   0.05      4.40
age          0.00 -2.61   0.01      6.79
race         0.00  1.02   0.31      1.70
wexp         0.00 -0.71   0.48      1.06
mar          0.00 -1.14   0.26      1.97
paro         0.00 -0.43   0.66      0.59
prio         0.00  3.19 <0.005      9.48
---
Concordance = 0.64
Partial AIC = 1331.50
log-likelihood ratio test = 33.27 on 7 df
-log2(p) of ll-ratio test = 15.37

## Cox Example Explanation

Targets
- week = survival duration
- arrest = whether the event (arrest) occurred (1) or was censored (0)
we're predicting risk of arrest at X week given features

Evaluations
- Concordance (like ROC-AUC)
- Partial log-likelihood (model fit-measure, lower = better)
- Partial AIC (penalized fit score, lower = better)
- Log-likelihood ratio test (overall model significance)

Features Interpretation
- coef: effect on the log hazard
- exp(coef): hazard ratio
- p: statistical significance (lower = more significance)

How do I use these from Cox to get these exactly?

Survival probability 
S(t∣X): the probability the patient survives beyond time 

Median survival time: the time at which 
S(t∣X)=0.5

Expected survival time (with care, since it’s semi-parametric)

- “What’s P(survive past t)?” → cph.predict_survival_function(X_new, times=[t])
- “What’s the median survival?” → cph.predict_median(X_new)
- “What’s the expected survival?” → compute RMST up to clinically meaningful tau via numeric integration of S(t|X). Use parametric AFT models if you require extrapolation beyond observed data.


Survival probability 
S(t∣X): the probability the patient survives beyond time 


Median survival time: the time at which 
S(t∣X)=0.5

Expected survival time (with care, since it’s semi-parametric)

In [ ]:
import numpy as np
import pandas as pd

def _prepare_times(events_time, events_observed):
    # returns sorted unique event times with indices where events occurred
    order = np.argsort(events_time)
    t_sorted = events_time[order]
    e_sorted = events_observed[order]
    return order, t_sorted, e_sorted

def fit_cox_ph(X, durations, events, max_iter=100, tol=1e-6, ridge=1e-6, verbose=False):
    """
    Fit CoxPH via Newton-Raphson with Breslow ties.
    X: (n, p) numpy array (no intercept)
    durations: (n,) times
    events: (n,) binary (1=event, 0=censor)
    Returns: dict with beta, var (covariance), baseline_survival (DataFrame)
    """
    n, p = X.shape
    # Ensure numpy arrays
    X = np.asarray(X, dtype=float)
    durations = np.asarray(durations, dtype=float)
    events = np.asarray(events, dtype=int)

    # sort by time ascending so risk sets are suffix sums (we'll compute reverse cumulative sums)
    order = np.argsort(durations)
    durations_s = durations[order]
    events_s = events[order]
    X_s = X[order]

    # initial beta
    beta = np.zeros(p, dtype=float)

    # unique event times and group indices for events
    unique_times, inv_idx = np.unique(durations_s[events_s == 1], return_inverse=True)
    # But for grouping we need mapping per event position; we'll compute group properties during iterations.

    for itr in range(max_iter):
        linpred = X_s.dot(beta)            # n
        # numeric stability for exp
        linpred_max = np.max(linpred)
        exp_lin = np.exp(linpred - linpred_max)  # shift to avoid overflow
        # compute cumulative sums over risk sets: since sorted ascending, risk set at time t_i is suffix starting at i
        # suffix sums can be computed via reversed cumulative sum
        exp_rev_cumsum = np.cumsum(exp_lin[::-1])[::-1]   # n
        # weighted X sums over risk sets
        X_weighted = X_s * exp_lin[:, None]
        Xw_rev_cumsum = np.cumsum(X_weighted[::-1, :], axis=0)[::-1, :]  # n x p

        # compute gradient and Hessian with Breslow for ties
        # iterate unique times where events occur; for efficiency, find indices where events==1 and group by durations
        event_idxs = np.where(events_s == 1)[0]
        times_of_events = durations_s[event_idxs]
        # group events by equal times
        groups = {}
        for idx, t in zip(event_idxs, times_of_events):
            groups.setdefault(t, []).append(idx)

        loglik = 0.0
        grad = np.zeros(p, dtype=float)
        hess = np.zeros((p, p), dtype=float)

        for t, idxs in groups.items():
            d = len(idxs)  # number of events at this time (ties)
            # sum of X over events at this time
            X_ev = X_s[idxs, :]           # d x p
            sum_X_ev = np.sum(X_ev, axis=0)  # p

            # risk set: those with time >= t: find first index where durations_s >= t
            # because durations_s sorted ascending, find first index i s.t. durations_s[i] >= t
            # using searchsorted
            i = np.searchsorted(durations_s, t, side='left')
            sum_risk = np.sum(exp_lin[i:])  # or exp_rev_cumsum[i]
            sum_X_risk = np.sum(X_weighted[i:, :], axis=0)  # or Xw_rev_cumsum[i]

            # Breslow approximation
            loglik += np.sum(linpred[idxs]) - d * (np.log(sum_risk) + linpred_max)  # note shifting back
            # gradient contribution
            grad += np.sum(X_ev, axis=0) - d * (sum_X_risk / sum_risk)
            # Hessian contribution:
            # V = sum_j (exp_lp_j * X_j X_j^T) / sum_risk  - (sum_X_risk/sum_risk) (sum_X_risk/sum_risk)^T
            sum_X2_risk = np.dot((X_s[i:].T * exp_lin[i:]), X_s[i:])  # p x p  (sum over j in risk of exp*X_j X_j^T)
            V = sum_X2_risk / sum_risk - np.outer(sum_X_risk / sum_risk, sum_X_risk / sum_risk)
            hess -= d * V

        # add ridge for stability
        hess_reg = hess - ridge * np.eye(p)  # note: Hessian is negative definite; we use -H in NR step
        # Newton-Raphson step: solve H * delta = grad  (since we maximize loglik, we do beta_new = beta - H^{-1} grad; note our hess is negative)
        try:
            delta = np.linalg.solve(hess_reg, grad)
        except np.linalg.LinAlgError:
            # fallback to pseudo-inverse
            delta = np.linalg.pinv(hess_reg).dot(grad)

        beta_new = beta - delta

        step = np.linalg.norm(beta_new - beta)
        beta = beta_new

        if verbose:
            print(f"iter {itr}: loglik={loglik:.6f}, ||delta||={step:.6g}")

        if step < tol:
            break

    # compute variance: negative inverse Hessian at solution
    # recompute Hessian at final beta
    linpred = X_s.dot(beta)
    linpred_max = np.max(linpred)
    exp_lin = np.exp(linpred - linpred_max)
    X_weighted = X_s * exp_lin[:, None]
    # recompute risk suffix sums
    exp_rev_cumsum = np.cumsum(exp_lin[::-1])[::-1]
    Xw_rev_cumsum = np.cumsum(X_weighted[::-1, :], axis=0)[::-1, :]

    # recompute hessian properly
    hess = np.zeros((p, p), dtype=float)
    event_idxs = np.where(events_s == 1)[0]
    times_of_events = durations_s[event_idxs]
    groups = {}
    for idx, t in zip(event_idxs, times_of_events):
        groups.setdefault(t, []).append(idx)

    for t, idxs in groups.items():
        d = len(idxs)
        i = np.searchsorted(durations_s, t, side='left')
        sum_risk = np.sum(exp_lin[i:])
        sum_X_risk = np.sum(X_weighted[i:, :], axis=0)
        sum_X2_risk = np.dot((X_s[i:].T * exp_lin[i:]), X_s[i:])  # p x p
        V = sum_X2_risk / sum_risk - np.outer(sum_X_risk / sum_risk, sum_X_risk / sum_risk)
        hess -= d * V

    # covariance = -inv(hess)
    try:
        cov = -np.linalg.inv(hess)
    except np.linalg.LinAlgError:
        cov = -np.linalg.pinv(hess)

    # baseline hazard and baseline survival (Breslow)
    # hazard increments at unique event times: delta_h(t) = d(t) / sum_{j in R(t)} exp(X_j beta)
    uniq_times = np.array(sorted(groups.keys()))
    baseline_hazard = []
    baseline_surv = []
    cum_h = 0.0
    for t in uniq_times:
        idxs = groups[t]
        d = len(idxs)
        i = np.searchsorted(durations_s, t, side='left')
        sum_risk = np.sum(np.exp(X_s.dot(beta) - linpred_max)[i:]) * np.exp(linpred_max)  # unshift
        # to avoid mixing shifted exponent, compute exp without shift for numerator/denom:
        sum_risk_raw = np.sum(np.exp(X_s.dot(beta))[i:])
        dh = d / sum_risk_raw
        cum_h += dh
        baseline_hazard.append((t, dh))
        baseline_surv.append((t, np.exp(-cum_h)))

    baseline_hazard_df = pd.DataFrame(baseline_hazard, columns=["time", "hazard"])
    baseline_surv_df = pd.DataFrame(baseline_surv, columns=["time", "S0"])

    # package results
    # reorder beta back to original order
    beta_full = beta.copy()
    # create helper predict functions
    def predict_partial_hazard(X_new):
        return np.exp(np.dot(X_new, beta_full))

    def predict_survival_function(X_new, times=None):
        # times: array-like of times. returns DataFrame index=times, columns = rows of X_new
        if times is None:
            times = baseline_surv_df["time"].values
        times = np.asarray(times)
        S0_t = np.interp(times, baseline_surv_df["time"].values, baseline_surv_df["S0"].values, left=1.0, right=baseline_surv_df["S0"].values[-1])
        ph = predict_partial_hazard(X_new)
        # S(t|X) = S0(t)^{exp(Xb)}
        out = np.vstack([S0_t ** ph_i for ph_i in ph])
        # transpose -> index times, columns per sample
        return pd.DataFrame(out.T, index=times, columns=np.arange(X_new.shape[0]))

    return {
        "beta": beta_full,
        "cov": cov,
        "baseline_hazard": baseline_hazard_df,
        "baseline_survival": baseline_surv_df,
        "predict_survival_function": predict_survival_function,
        "predict_partial_hazard": predict_partial_hazard,
    }

# -------------------------
# Quick usage example (synthetic)
if __name__ == "__main__":
    # small synthetic example
    np.random.seed(0)
    n = 200
    p = 3
    X = np.random.randn(n, p)
    true_beta = np.array([0.5, -0.7, 0.0])
    lin = X.dot(true_beta)
    base_hazard = 0.01
    # simulate exponential survival times with hazards h0 * exp(Xb)
    U = np.random.rand(n)
    times = -np.log(U) / (base_hazard * np.exp(lin))
    # censor some at t=50
    censoring = np.random.exponential(scale=100, size=n)
    durations = np.minimum(times, censoring)
    events = (times <= censoring).astype(int)

    res = fit_cox_ph(X, durations, events, max_iter=50, tol=1e-6, verbose=True)
    print("beta:", res["beta"])
    print("cov diag:", np.sqrt(np.diag(res["cov"])))
    # predict survival for first two rows
    sf = res["predict_survival_function"](X[:2, :], times=np.linspace(0, np.max(durations), 50))
    print(sf.head())
